In [101]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import LabelEncoder
from sklearn.naive_bayes import GaussianNB

In [102]:
class NaiveBayes:
    def __init__(self, csv_file):
        self.csv_file = csv_file
        self.df = None
        self.feature_probabilities = None

    def csv_reader(self):
        self.df = pd.read_csv(self.csv_file)  
        return self.df

    def total_elements(self):
        return self.df.shape[0]

    def calculate_yes_no(self):
        counts = self.df['buys_computer'].value_counts() 
        total_yes = counts.get("yes", 0)
        total_no = counts.get("no", 0)
        return total_yes, total_no


    def calculate_feature_probabilities(self):
        total_yes, total_no = self.calculate_yes_no()
        feature_probabilities = {}
        
        for feature in ['age', 'income', 'student', 'credit_rating']:
            feature_probabilities[feature] = {}
            
            for category in self.df[feature].unique():
                count_yes = len(self.df[(self.df[feature] == category) & (self.df['buys_computer'] == 'yes')])
                count_no = len(self.df[(self.df[feature] == category) & (self.df['buys_computer'] == 'no')])
                
                prob_yes = count_yes / total_yes if total_yes > 0 else 0
                prob_no = count_no / total_no if total_no > 0 else 0
                
                feature_probabilities[feature][category] = {'yes': prob_yes, 'no': prob_no}
        
        self.feature_probabilities = feature_probabilities
        return feature_probabilities

    def probability_total_yes_no(self):
        total_yes, total_no = self.calculate_yes_no()
        total_elements = self.total_elements()
        prob_y = total_yes / total_elements if total_elements > 0 else 0
        prob_n = total_no / total_elements if total_elements > 0 else 0
        
        return prob_y, prob_n

    def calculate_probability(self, input_data):
        prob_yes = 1.0
        prob_no = 1.0
        
        for feature, category in input_data.items():
            if feature in self.feature_probabilities:
                prob_yes *= self.feature_probabilities[feature].get(category, {}).get('yes', 0)
                prob_no *= self.feature_probabilities[feature].get(category, {}).get('no', 0)
        prob_y, prob_n = self.probability_total_yes_no()
        p_y = prob_yes * prob_y
        p_n = prob_no * prob_n
        
        return p_y, p_n

    def final_result(self, input_data):
        if self.feature_probabilities is None:
            raise ValueError("Feature probabilities have not been calculated.")
        
        prob_yes, prob_no = self.calculate_probability(input_data)
        
        if prob_yes > prob_no:
            print(f"The prediction is Yes with probability {prob_yes}")
        elif prob_no > prob_yes:
            print(f"The prediction is No with probability {prob_no}")
        else:
            print("The prediction is inconclusive.")

In [103]:
def main(input_data):
    nb = NaiveBayes('student.csv')
    nb.csv_reader() 
    nb.calculate_feature_probabilities()  
    nb.final_result(input_data)

In [104]:
if __name__ == "__main__":
    input_data = {
        'age': 'youth',
        'income': 'medium_aged',
        'student': 'no',
        'credit_rating': 'fair'
    }
    
    main(input_data)

KeyError: 'buys_computer'